<a href="https://colab.research.google.com/github/verityw/manipulation-final-project/blob/main/MaskRCNN_and_ICP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clutter_gen setup

In [1]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='latest', drake_build='continuous')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Setup rendering (with xvfb), if necessary:
import os
if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start two meshcat servers (one for 2d) to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc_2d, zmq_url_2d, web_url_2d = start_zmq_server_as_subprocess(server_args=server_args)
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Imports
import numpy as np
from IPython.display import display, HTML
from ipywidgets import Textarea

from pydrake.all import ( 
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, 
    DiagramBuilder, RigidTransform, RotationMatrix, Box,    
    CoulombFriction, FindResourceOrThrow, FixedOffsetFrame, 
    GeometryInstance, MeshcatContactVisualizer, Parser, PlanarJoint,  
    JointIndex, Simulator, ProcessModelDirectives, LoadModelDirectives
)

from functools import partial
import open3d as o3d
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

from pydrake.all import (
    ConnectPlanarSceneGraphVisualizer,
    ConnectDrakeVisualizer, DepthCameraProperties, RgbdSensor,
    RandomGenerator, UniformlyRandomRotationMatrix, RollPitchYaw,
    MakeRenderEngineVtk, RenderEngineVtkParams, Role, UnitInertia, set_log_level
)

from ipywidgets import Dropdown, FloatSlider, Layout
from pydrake.all import (
    Sphere, Cylinder, Box, Capsule, Ellipsoid, SpatialInertia)

import pydrake

#from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.meshcat_utils import draw_open3d_point_cloud, draw_points
from manipulation.open3d_utils import create_open3d_point_cloud
from manipulation.mustard_depth_camera_example import MustardExampleSystem
from manipulation.scenarios import AddRgbdSensors
from manipulation.utils import FindResource

set_log_level("warn");

ycb = [("cracker", "003_cracker_box.sdf"), 
        ("sugar", "004_sugar_box.sdf"), 
        ("soup", "005_tomato_soup_can.sdf"), 
        ("mustard", "006_mustard_bottle.sdf"), 
        ("gelatin", "009_gelatin_box.sdf"), 
        ("meat", "010_potted_meat_can.sdf")]


ICP Setup

In [2]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='a43d49331e92aa64ca7dbe21882ef5853218753c', drake_version='20200921', drake_build='nightly')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Setup rendering (with xvfb), if necessary:
import os
if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
server_args = []
if 'google.colab' in sys.modules:
    server_args = ['--ngrok_http_tunnel']
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
import meshcat
import open3d as o3d
import meshcat.geometry as g
import meshcat.transformations as tf

from pydrake.all import (
    AddMultibodyPlantSceneGraph, BaseField, Box, ConnectMeshcatVisualizer, DepthCameraProperties, 
    DepthImageToPointCloud, DiagramBuilder, FindResourceOrThrow, MakeRenderEngineVtk, CameraInfo, PixelType, 
    MeshcatPointCloudVisualizer, Parser, RenderEngineVtkParams, RgbdSensor, RigidTransform, Simulator,
    RollPitchYaw, RotationMatrix, SpatialInertia, LeafSystem, AbstractValue, PointCloud, Fields)

ICP Util functions

In [3]:
"""
Point Cloud Utilities. Converts between three forms of pointcloud objects:
  - Drake Pointcloud Object
  - Open3d Pointcloud Object
  - Nx3 numpy array
"""

def pcl_np2o3d(pcl_np):
  """
  Input: Nx3 np array of points.
  Ouput: Open3d np object. 
  """
  assert(pcl_np.shape[1] == 3) # sanity check
  pcl_o3d = o3d.geometry.PointCloud()
  pcl_o3d.points = o3d.utility.Vector3dVector(pcl_np)
  return pcl_o3d 

def pcl_o3d2np(pcl_o3d):
  """
  Input: Open3d np object.
  Output: Nx3 np array of points. 
  """
  return np.asarray(pcl_o3d.points)
  
def pcl_np2drake(pcl_np, color):
  """
  Input: Nx3 np array of points in mm, and and 3x1 np array of color using uin8 format.
  Output: drake Pointcloud object. 
  """
  assert(pcl_np.shape[1] == 3) # sanity check. 
  pcl_drake = PointCloud(new_size = pcl_np.shape[0],
                         fields= Fields(BaseField.kXYZs | BaseField.kRGBs))
  xyzs = pcl_drake.mutable_xyzs()
  xyzs[:,:] = np.array(pcl_np).transpose()
  rgbs = pcl_drake.mutable_rgbs()
  rgbs[:,:] = np.tile(color, (pcl_np.shape[0], 1)).transpose()
  return pcl_drake

def pcl_drake2np(pcl_drake):
  """
  Input: drake Pointcloud object.
  Output: Nx3 np array of points. 
  """
  xyzs = pcl_drake.mutable_xyzs().copy().transpose()
  return xyzs 

def rgb_drake2np(pcl_drake):
  rgbs = pcl_drake.mutable_rgbs().copy().transpose()
  return rgbs 

def visualize_pointcloud(vis, pcl, name="point_cloud", X_WC=RigidTransform()):
  """
  pcl: Drake pointcloud object. 
  All other formats should convert to this format before passing to visualization. 
  """
  vis[name].set_object(g.PointCloud(X_WC.multiply(pcl.xyzs()), pcl.rgbs() / 255., size=0.001))

def visualize_pointcloud_xyzrgb(vis, pcl_xyz, pcl_rgb, name="point_cloud", X_WC=RigidTransform()):
  """
  Visualization function to access Meshcat's method more directly. 
  """
  vis[name].set_object(g.PointCloud(X_WC.multiply(pcl_xyz), pcl_rgb / 255., size=0.001))

Get scene (WILL EVENTUALLY BE REPLACED BY A MODIFICATION OF clutter_gen)

In [17]:
def setup_clutter_station(X_WC):
  builder = DiagramBuilder()
    
  plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)

  parser = Parser(plant)

  parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/bin.sdf"))
  plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("bin_base"))


  rs = np.random.RandomState()  # this is for python
  generator = RandomGenerator(rs.randint(1000))  # this is for c++

  # Only generate a few objects for testing.
  for i in [2]: #[2, 3]:
      object_num = i
      sdf = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/" + ycb[object_num][1])
      parser.AddModelFromFile(sdf, f"object{i}")
  # Add a rendering engine
  renderer = "my_renderer"
  scene_graph.AddRenderer(renderer,
                          MakeRenderEngineVtk(RenderEngineVtkParams()))
  
  plant.Finalize() 
  plant.GetFloatingBaseBodies() 
  frames_to_draw = {"cameras": "camera"}
  meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
                                      delete_prefix_on_load=False,
                                      frames_to_draw=frames_to_draw)
  meshcat.load() 

  properties = DepthCameraProperties(width=640,
                                      height=480,
                                      fov_y=np.pi / 4.0,
                                      renderer_name=renderer,
                                      z_near=0.1,
                                      z_far=10.0)
  camera = builder.AddSystem(
        RgbdSensor(parent_id=scene_graph.world_frame_id(),
                   X_PB=RigidTransform(
                       RollPitchYaw(np.pi, 0, np.pi/2.0),
                       [0, 0, .8]),
                   properties=properties,
                   show_window=False))
  camera.set_name("rgbd_sensor")
  builder.Connect(scene_graph.get_query_output_port(),
                  camera.query_object_input_port())

  # Export the camera outputs
  builder.ExportOutput(camera.color_image_output_port(), "color_image")
  builder.ExportOutput(camera.depth_image_32F_output_port(), "depth_image")

  # Add a system to convert the camera output into a point cloud
  to_point_cloud = builder.AddSystem(
      DepthImageToPointCloud(camera_info=camera.depth_camera_info(),
                              pixel_type=PixelType.kDepth32F,
                              fields=BaseField.kXYZs | BaseField.kRGBs))
  builder.Connect(camera.depth_image_32F_output_port(),
                  to_point_cloud.depth_image_input_port())
  builder.Connect(camera.color_image_output_port(),
                  to_point_cloud.color_image_input_port())

  # Send the point cloud to meshcat for visualization, too.
  meshcat_pointcloud = builder.AddSystem(MeshcatPointCloudVisualizer(meshcat, X_WP=X_WC, draw_period=1./5.))
  builder.Connect(to_point_cloud.point_cloud_output_port(), meshcat_pointcloud.get_input_port())

  # Export the point cloud output.
  builder.ExportOutput(to_point_cloud.point_cloud_output_port(),
                        "point_cloud")

  diagram = builder.Build()
  diagram.set_name("depth_camera_demo_system")
  return diagram, plant, generator, rs, builder, scene_graph 

# Set pose of the brick 
X_WO = RigidTransform(RollPitchYaw(0, 0, np.pi/5).ToRotationMatrix(), np.array([-0.1, -0.6, 0.09]))

# Setup CameraPose
X_WC = RigidTransform(
                       RollPitchYaw(np.pi, 0, np.pi/2.0),
                       [0, 0, .8])

# Take a pointcloud snapshot of the scene with the brick. 
diagram, plant, generator, rs, builder, scene_graph = setup_clutter_station(X_WC)
print("OUTSIDE setup_clutter_station")
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(context)
transforms = []
z = 0.1
for body_index in plant.GetFloatingBaseBodies():
    print(plant.get_body(body_index))
    tf = RigidTransform(
            UniformlyRandomRotationMatrix(generator),  
            [rs.uniform(-.15,.15), rs.uniform(-.2, .2), z])
    print(tf.GetAsMatrix4())
    plant.SetFreeBodyPose(plant_context, 
                          plant.get_body(body_index),
                          tf)
    z += 0.1
    transforms.append(tf)
scene_pcl_drake = diagram.GetOutputPort("point_cloud").Eval(context)
scene_pcl_np = X_WC.multiply(pcl_drake2np(scene_pcl_drake).transpose()).transpose()
scene_rgb_np = rgb_drake2np(scene_pcl_drake)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6002...
You can open the visualizer by visiting the following URL:
http://0d6c368c0be6.ngrok.io/static/
Connected to meshcat-server.
OUTSIDE setup_clutter_station
<RigidBody_[float] name='base_link_soup' index=2 model_instance=3>
[[-0.61347646 -0.30826448  0.72706234 -0.03373933]
 [-0.55074214 -0.4928238  -0.67365257 -0.18704555]
 [ 0.56597679 -0.81369387  0.13256156  0.1       ]
 [ 0.          0.          0.          1.        ]]


Visualize setup


In [20]:
vis = meshcat.Visualizer(zmq_url).open()
vis.delete()

visualize_pointcloud_xyzrgb(vis, 
                            scene_pcl_np.transpose(), 
                            scene_rgb_np.transpose(), 
                            name="pcl_scene_background")

You can open the visualizer by visiting the following URL:
http://0d6c368c0be6.ngrok.io/static/
